In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sklearn
import sys
import tensorflow as tf
from tensorflow import keras
import random
import cv2
import seaborn

In [ ]:
imdb = keras.datasets.imdb
vocab_size = 10000
index_from = 3
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(
    num_words = vocab_size, index_from = index_from)

In [ ]:
word_len = [len(sentence) for sentence in train_data]
print("max len: {} min len: {} mean len: {}".format(max(word_len), min(word_len), sum(word_len) // len(word_len)))

In [ ]:
word_index = imdb.get_word_index()
word_index = {k:(v+3) for k, v in word_index.items()}

In [ ]:
word_index['<PAD>'] = 0
word_index['<START>'] = 1
word_index['<UNK>'] = 2
word_index['<END>'] = 3

In [ ]:
index_word = dict(
    [(value, key) for key, value in word_index.items()])

def decode_review(text_ids):
    return ' '.join(
        [index_word.get(word_id, "<UNK>") for word_id in text_ids])

In [ ]:
decode_review(train_data[0])

In [ ]:
max_length = 500

train_data = keras.preprocessing.sequence.pad_sequences(
    train_data, # list of list
    value = word_index['<PAD>'],
    padding = 'post', # post, pre
    maxlen = max_length)

test_data = keras.preprocessing.sequence.pad_sequences(
    test_data, # list of list
    value = word_index['<PAD>'],
    padding = 'post', # post, pre
    maxlen = max_length)

# print(decode_review(train_data[0]))

In [ ]:
embedding_dim = 16
batch_size = 1024

single_rnn_model = keras.models.Sequential([
    # 1. define matrix: [vocab_size, embedding_dim]
    # 2. [1,2,3,4..], max_length * embedding_dim
    # 3. batch_size * max_length * embedding_dim
    keras.layers.Embedding(vocab_size, embedding_dim,
                           input_length = max_length,
                           mask_zero=True),
    keras.layers.SimpleRNN(units = 64, return_sequences = False),
    keras.layers.Dense(64, activation = 'relu'),
    keras.layers.Dense(1, activation='sigmoid'),
])

single_rnn_model.summary()
single_rnn_model.compile(optimizer = 'adam',
                         loss = 'binary_crossentropy',
                         metrics = ['accuracy'])

In [ ]:
earlyStop = keras.callbacks.EarlyStopping(patience=3)
epochs = 20
history_single_rnn = single_rnn_model.fit(
    train_data, train_labels,
    epochs = epochs,
    batch_size = batch_size,
    validation_split = 0.2,
    callbacks = [earlyStop])

In [ ]:
def plot_learning_curves(history, label):
    data = {}
    data[label] = history.history[label]
    data['val_'+label] = history.history['val_'+label]
    pd.DataFrame(data).plot(figsize=(8, 5))
    plt.grid(True)
    plt.show()
    
plot_learning_curves(history_single_rnn, 'accuracy')
plot_learning_curves(history_single_rnn, 'loss')

In [ ]:
bi_rnn_model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim,
                           input_length = max_length,
                           mask_zero=True),
    keras.layers.Bidirectional(
        keras.layers.SimpleRNN(
            units = 32, return_sequences = False)),
    keras.layers.Dense(32, activation = 'relu'),
    keras.layers.Dense(1, activation='sigmoid'),
])

bi_rnn_model.summary()
bi_rnn_model.compile(optimizer = 'adam',
                     loss = 'binary_crossentropy',
                     metrics = ['accuracy'])

In [ ]:
history_bi_rnn = bi_rnn_model.fit(
    train_data, train_labels,
    epochs = 30,
    batch_size = batch_size,
    validation_split = 0.2,
    callbacks = [earlyStop])

In [ ]:
plot_learning_curves(history_bi_rnn, 'accuracy')
plot_learning_curves(history_bi_rnn, 'loss')

In [ ]:
bi_rnn_dropout_model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim,
                           input_length = max_length,
                           mask_zero=True),
    keras.layers.Bidirectional(
        keras.layers.SimpleRNN(
            units = 32, return_sequences = False)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(32, activation = 'relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid'),
])

bi_rnn_dropout_model.summary()
bi_rnn_dropout_model.compile(optimizer = 'adam',
                             loss = 'binary_crossentropy',
                             metrics = ['accuracy'])

In [ ]:
history_bi_rnn_dropout = bi_rnn_dropout_model.fit(
    train_data, train_labels,
    epochs = 30,
    batch_size = batch_size,
    validation_split = 0.2,
    callbacks = [earlyStop])

In [ ]:
plot_learning_curves(history_bi_rnn_dropout, 'accuracy')
plot_learning_curves(history_bi_rnn_dropout, 'loss')

In [ ]:
initializer = keras.initializers.Orthogonal()
bi_lstm_model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim,
                           input_length = max_length,
                           mask_zero=True),
    keras.layers.Bidirectional(
        keras.layers.LSTM(
            units = 32, return_sequences = False, kernel_initializer=initializer)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(32, activation = 'selu', kernel_initializer=initializer),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid'),
])

bi_lstm_model.summary()
bi_lstm_model.compile(optimizer = keras.optimizers.Adam(1e-4),
                             loss = 'binary_crossentropy',
                             metrics = ['accuracy'])

In [ ]:
history_bi_lstm = bi_lstm_model.fit(
    train_data, train_labels,
    epochs = 50,
    batch_size = batch_size,
    validation_split = 0.2,
    callbacks = [earlyStop])

In [ ]:
plot_learning_curves(history_bi_rnn_dropout, 'accuracy')
plot_learning_curves(history_bi_rnn_dropout, 'loss')